In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class TreeNode():
    def __init__(self,indices, depth, feature_ind,entropy,side, majority, left_c=None, right_c=None ):
        
        
        self.indices = indices  #indices of data of the Node
        
        self.depth = depth
        self.feature_ind = feature_ind
        
        self.entropy = entropy
        self.side = side 
        
        self.left_c = left_c #TreeNode obj
        self.right_c = right_c #TreeNode obj
        self.majority = majority
        
        
    def __str__(self):
        
        return "{} DEPTH={}, feature={}, majority={}, entropy = {}".format(self.side, 
                                                                            self.depth,
                                                                            self.feature_ind,
                                                                            self.majority,
                                                                            self.entropy
                                                                          )
    
    
    

In [3]:
class DecisionTree():
    
    def __init__(self, X,y, max_depth, feature_names=[]): #X-matrix, y-array
        
        
        self.lines, self.feature_nums = X.shape
        
        
        
        self.indices = [i for i in range(len(y))] 
        self.X = X
        self.y = y
        self.max_depth = max_depth
        
        if feature_names == []:
            self.feature_names = ["feature {}".format(i) for i in range(X.shape[0])]
        else:
            self.feature_names = feature_names
            
            
        
        self.ROOT_NODE = None #Top of the tree
        
        
    
    def get_sample(self, indices):
        
        
        m,n = len(indices), self.X.shape[1]
        
        X_sample = np.zeros((m,n))
        y_sample = []
        
        
        for i in range(len(indices)):
            
            for j in range(n):
                
                    
                X_sample[i,j] = self.X[indices[i],j]
                
                
                            
            y_sample.append(self.y[indices[i]])
            
            
            
        return X_sample, y_sample
    
    
            
    def split_data(self, indices, feature_ind):
        
        
        left_indices = []
        right_indices = []
        
        
        for i in indices:
            
            if self.X[i, feature_ind] == 1:
                left_indices.append(i)
                
            else:
                right_indices.append(i)
                
                
        
        return left_indices, right_indices
    
    
    def entropy(self, y): #use y directly
        
        if len(y) == 0:
            return 0
        
        
        n_positive = np.sum(y)
        total = len(y)
        
        
        p = n_positive / total
        
        if p == 0 or p == 1:
            return 0
        
        
        return -p*np.log2(p) - (1-p)*np.log2(1-p)
    
    
    
    def info_gain(self, node_indices, feature_ind):
        
        y = self.y[node_indices]
        
        left_indices, right_indices = self.split_data(node_indices, feature_ind)
        
        
        parent_entropy = self.entropy(y)
        left_entropy = self.entropy(self.y[left_indices])
        right_entropy = self.entropy(self.y[right_indices])
        
        #calculate weights
        
        w_left = len(self.y[left_indices]) / len(self.y[node_indices])
        w_right = len(self.y[right_indices]) / len(self.y[node_indices])
        
        
        inf_gain = parent_entropy - (w_left*left_entropy + w_right*right_entropy)
        
        return inf_gain
    
    
    def get_majority(self, indices):
        
        _ , y_data = self.get_sample(indices)
        
        #compute ratio
        
        ratio = np.sum(y_data) / len(y_data)
        
        if ratio >= 0.5:
            return 1
        else:
            return 0
        
        
        
        
    
    def get_best_split(self, node_indices):
        
        if self.entropy(node_indices) == 0:
            #Already perfect
            return None
        
        
        best_feature_ind = -1
        best_info_gain = 0
        
        
        for feature in range(self.feature_nums):
            
            inf_g = self.info_gain(node_indices, feature)
            
            if inf_g > best_info_gain:
                
                #update
                best_feature_ind = feature
                best_info_gain = inf_g
                
                
        return best_feature_ind, best_info_gain
    
    
    def build_tree(self, node=None, curr_depth=0):
        #BASE CASES
        
        if curr_depth > self.max_depth: #exceed limit
            return
        
        
        
        
         
        
        #initialize if node is None
        if node is None:
            
            r_inds = self.indices
            r_depth = curr_depth
            r_feature_ind, _ =  self.get_best_split(r_inds)
            
            if r_feature_ind is None:
                return
            
            
            r_side = "root"
            r_entropy = self.entropy(self.y[r_inds])
            r_major = self.get_majority(r_inds)
            
            
            
            
            
            
            r_node = TreeNode(r_inds, 
                            r_depth, r_feature_ind,
                            r_entropy, r_side, r_major
                           
                           )
            
            self.ROOT_NODE = r_node
            
            #run recursive call for root node
            
            self.build_tree(r_node, curr_depth)
            
            
            return r_node
        
        if self.entropy(node.indices) == 0: #already pure
            return
            
            
            
            
        #RECURSIVE CASE

        best_feature = self.get_best_split(node.indices)
        
        if best_feature is None:
            return
        
        
        

        left_ind, right_ind = self.split_data(node.indices, best_feature[0])
        
        #compute left  and right majorities
        left_maj = self.get_majority(left_ind)
        right_maj = self.get_majority(right_ind)
        
        
        

        #Creating new TreeNodes

        l_node = TreeNode(

            left_ind,  
            node.depth + 1,  best_feature[0],
            self.entropy(self.y[left_ind]), "left", left_maj


        )


        r_node = TreeNode(

            right_ind,   
            node.depth + 1,  best_feature[0],
            self.entropy(self.y[right_ind]), "right", right_maj


        )

        
        node.left_c = self.build_tree(l_node, curr_depth + 1)
        node.right_c = self.build_tree(r_node, curr_depth + 1)
        
        return node

    
    def _predict_instance(self, node ,instance): #input - array with features vals
        
        if node.entropy == 0:
            
            return node.majority
        
        
        #feature to go
        feature = node.feature_ind
        
        if instance[feature] == 1: #go left
            
            if node.left_c != None: #check if there is a left child
                
                return self._predict_instance(node.left_c, instance)
            
            else:
                return node.majority
            
        else: #go right
            
            if node.right_c != None:
                
                return self._predict_instance(node.right_c, instance)
            else:
                return node.majority
            
            
            
    def predict(self, X_test):
        #X_test - ndarray or list of lists
        
        predictions = []
        
        
        for i in range(len(X_test)):
            
            res = self._predict_instance(self.ROOT_NODE,  X_test[i])
            
            predictions.append(res)
            
            
        return predictions
            
        




In [4]:
# Features
X = np.array([
    [1, 0, 1],
    [0, 1, 0],
    [1, 1, 1],
    [1, 0, 0],
    [0, 1, 1],
    [0, 0, 0],
    [1, 1, 0],
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 0],
    [1, 1, 1],
    [1, 0, 0],
    [0, 1, 1],
    [0, 0, 0],
    [1, 1, 0],
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 0],
    [1, 1, 1],
    [1, 0, 0],
    [0, 1, 1],
    [0, 0, 0],
    [1, 1, 0],
    [0, 0, 1]
])

# Labels
y = np.array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0])

# Feature names
feature_names = ['Feature 1', 'Feature 2', 'Feature 3']

# Initialize Decision Tree
dt = DecisionTree(X, y, max_depth=3, feature_names=feature_names)

# Build the tree
dt.build_tree()

preds = dt.predict(X)

/tmp/ipykernel_4613/536170397.py:88: RuntimeWarning: invalid value encountered in log2
  return -p*np.log2(p) - (1-p)*np.log2(1-p)
/tmp/ipykernel_4613/536170397.py:120: RuntimeWarning: invalid value encountered in double_scalars
  ratio = np.sum(y_data) / len(y_data)


In [5]:
#Let's test how it managed with our simple syntetic data


from sklearn.metrics import accuracy_score


print("Accuracy score = ", accuracy_score(preds, y))

Accuracy score =  1.0


## It is still not perfect, but it covers basic idea of Decision Tree